In [1]:
from pathlib import Path

import polars as pl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
data_pth = Path('../data/')

In [174]:
time_intervals = pl.read_csv(data_pth/ 'grade_timesteps.csv')

In [188]:
# time_intervals = pl.read_csv(data_pth/ 'time_intervals.csv')

In [189]:
time_intervals_final = time_intervals.group_by('id').agg(pl.col('dt').diff().mean())


In [3]:
df = pl.read_csv(data_pth/'graded_data.csv')
area_delta = df[['id','area']].group_by('id').agg(pl.col('area').max()-pl.col('area').min())

large_blasto_mask = area_delta['area']>8000

selected_ids = area_delta.filter(large_blasto_mask.to_list())['id']

final_df = df.filter(df["id"].is_in(selected_ids))']
    

In [112]:
time_intervals.group_by('id').agg(pl.col('dt').diff().mean()).max()['dt']

dt
f64
0.010597


In [66]:
time_intervals.group_by('id').agg(pl.col('dt').diff().mean())

id,dt
str,f64
"""D2016.06.01_S1327_I149_7""",0.007032
"""D2016.07.16_S1374_I149_4""",0.006986
"""D2016.06.24_S1348_I149_4""",0.007007
"""D2016.02.22_S1235_I149_6""",0.007009
"""D2016.01.05_S1181_I149_4""",0.006967
…,…
"""D2016.06.24_S1348_I149_8""",0.007007
"""D2016.02.25_S1241_I149_9""",0.007057
"""D2016.02.13_S1224_I149_1""",0.006983


In [123]:
from scipy.signal import find_peaks
def get_inflection_points(area):
    
    x = np.arange(len(area.to_numpy()))
    y = area  # Your data here


    # Compute the first and second derivatives
    dy = np.gradient(y)
    d2y = np.gradient(dy)

    # Set a threshold for the second derivative
    threshold = np.max(d2y) * 0.5

    sharp_points, _ = find_peaks(d2y, height=threshold)
    
    return sharp_points

In [144]:
time_intervals_final.filter(pl.col("id") == "D2016.07.02_S1360_I149_3")['dt'].item().seconds/60

10.083333333333334

In [124]:
time_intervals_final

id,dt
str,duration[μs]
"""D2016.07.02_S1360_I149_3""",10m 5s 600804µs
"""D2016.10.18_S1418_I149_9""",15m 1s 502953µs
"""D2016.02.25_S1241_I149_2""",10m 9s 681058µs
"""D2016.04.19_S1290_I149_8""",10m 731982µs
"""D2016.03.12_S1256_I149_4""",10m 9s 750677µs
…,…
"""D2016.07.02_S1359_I149_7""",10m 6s 149155µs
"""D2017.09.21_S1624_I149_2""",15m 2s 956941µs
"""D2016.05.16_S1310_I149_4""",10m 10s 44928µs


In [157]:
4*24

96

In [208]:
import polars as pl
from multiprocessing import Pool, cpu_count

# Example function
# def get_inflection_points(area_series):
#     # Perform computations here
#     return some_result

# Parallel processing wrapper
def process_group(group):
    group_id, group_data, embryo_dt = group
    
    if embryo_dt.is_empty():
        return group_id, 0

    if "dt" not in embryo_dt.columns:
        raise ValueError(f"Column 'dt' not found in time_intervals_final for id {group_id[0]}")

    # Compute peaks
    peaks = get_inflection_points(group_data["area"])

    # Ensure dt_in_hours is a scalar or iterable
    dt_in_hours = embryo_dt["dt"].item()*1440/60  # Convert to list

    # if len(dt_in_hours) != 1:
    #     raise ValueError(f"Expected one dt value per group but found {len(dt_in_hours)} for id {group_id[0]}")

    # Extract the scalar value for dt
    dt_in_hours_scalar = dt_in_hours

    # Filter peaks
    ok_peaks = [peak for peak in peaks if peak * dt_in_hours > 96]

    # Return the minimum valid peak
    return group_id, min(ok_peaks, default=None)

# Extract groups, process in parallel, and reconstruct
def parallel_map_groups(df, group_column, time_intervals_final):
    grouped = df.group_by(group_column, maintain_order=True)
    groups = [
        (group_id, group_data, time_intervals_final.filter(pl.col("id") == group_id[0]))
        for group_id, group_data in grouped
    ]
    
    # Set process count based on available cores
    num_processes = min(len(groups), cpu_count())

    with Pool(processes=num_processes) as pool:
        results = pool.map(process_group, groups)
    
    # Filter out groups with no valid peaks
    results = [(group_id, corr) for group_id, corr in results if corr is not None]
    
    # Rebuild the dataframe
    return pl.DataFrame({"id": [result[0][0] for result in results], "peak": [result[1] for result in results]})

# Apply to your data
peaks_area = parallel_map_groups(final_df, "id", time_intervals_final)



In [209]:
final_df.filter(pl.col('id')=='D2016.01.11_S1183_I149_4')['area'].shape

(685,)

In [212]:
peaks_area.write_csv(data_pth/'area_peaks.csv')